In [1]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'big_5_leagues_odds_5-11.csv'
big_df = pd.read_csv(file_path)
# Rename columns in big_df
big_df = big_df.rename(columns={
    'Combined Projected Home Goals': 'Projected Home Goals',
    'Combined Projected Away Goals': 'Projected Away Goals',
    'Combined Total Projected Goals': 'Projected Total Goals'
})

# Display the first few rows to verify the changes
print(big_df.head())


# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(big_df.head())


                Game Matchup  Projected Home Goals  Projected Away Goals  \
0          Alaves vs. Girona                  1.14                  0.96   
1  Athletic Club vs. Osasuna                  1.26                  0.73   
2    Granada vs. Real Madrid                  0.86                  1.28   
3    Mallorca vs. Las Palmas                  1.00                  0.63   
4     Villarreal vs. Sevilla                  1.36                  1.04   

   Projected Total Goals Home Win Probability Away Win Probability  \
0                   2.10               38.96%               31.63%   
1                   1.99               48.94%               20.92%   
2                   2.14               25.20%               47.06%   
3                   1.63               42.74%               22.97%   
4                   2.41               44.80%               28.31%   

  Draw Probability  Implied Home Odds  Implied Away Odds  Implied Draw Odds  \
0           29.41%                156      

In [2]:
import pandas as pd

# Example setup (replace with your actual DataFrame loading or definition)
# big_df = pd.read_csv('your_file.csv') or however you usually load or define your DataFrame

# Define the conversion function from American to Decimal odds
def american_to_decimal(american_odds):
    if pd.isna(american_odds):
        return None  # Handle NaN values if any
    if american_odds > 0:
        return american_odds / 100 + 1
    else:
        return 100 / abs(american_odds) + 1

# List of specific columns to convert
odds_columns = [
    'Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds',
    'Home Odds', 'Away Odds', 'Draw Odds'
]

# Convert the specified odds columns from American to Decimal odds
for column in odds_columns:
    if column in big_df.columns:  # Check if the column exists in the DataFrame
        big_df[column] = big_df[column].apply(american_to_decimal)

# Now big_df has the specified odds columns converted to decimal odds
# Optionally, you can view or export your DataFrame to verify the changes
print(big_df.head())  # Print the first few rows to check some of the conversions

# If you need to save this updated DataFrame:
# big_df.to_csv('updated_file.csv', index=False)  # Save to CSV without the index


                Game Matchup  Projected Home Goals  Projected Away Goals  \
0          Alaves vs. Girona                  1.14                  0.96   
1  Athletic Club vs. Osasuna                  1.26                  0.73   
2    Granada vs. Real Madrid                  0.86                  1.28   
3    Mallorca vs. Las Palmas                  1.00                  0.63   
4     Villarreal vs. Sevilla                  1.36                  1.04   

   Projected Total Goals Home Win Probability Away Win Probability  \
0                   2.10               38.96%               31.63%   
1                   1.99               48.94%               20.92%   
2                   2.14               25.20%               47.06%   
3                   1.63               42.74%               22.97%   
4                   2.41               44.80%               28.31%   

  Draw Probability  Implied Home Odds  Implied Away Odds  Implied Draw Odds  \
0           29.41%               2.56      

In [3]:
import pandas as pd
from itertools import combinations
import random

def expand_bets(row):
    results = []
    if row['Home Win Edge'] > 0.15:
        results.append((row['Game Matchup'], 'Home', row['Home Odds'], row['Home Win Edge']))
    if row['Away Win Edge'] > 0.15:
        results.append((row['Game Matchup'], 'Away', row['Away Odds'], row['Away Win Edge']))
    if row['Draw Edge'] > 0.15:
        results.append((row['Game Matchup'], 'Draw', row['Draw Odds'], row['Draw Edge']))
    return results

# Sample data loading and processing
# Replace this with actual DataFrame loading code
# Assuming 'big_df' is your DataFrame
expanded_bets = []
big_df.apply(lambda row: expanded_bets.extend(expand_bets(row)), axis=1)
df_expanded = pd.DataFrame(expanded_bets, columns=['Game Matchup', 'Edge Type', 'Odds', 'Edge'])

# Shuffle for random distribution
random.shuffle(expanded_bets)

# Define function to create round robins
def create_round_robins(entries, n, count):
    results = []
    usage_count = {}
    for entry in entries:
        key = (entry[0], entry[1])  # Game Matchup and Edge Type
        if key not in usage_count:
            usage_count[key] = 0

    for _ in range(count):
        current_combo = []
        for entry in entries:
            key = (entry[0], entry[1])
            if usage_count[key] < 3 and len(current_combo) < n:
                current_combo.append(entry)
                usage_count[key] += 1
                if len(current_combo) == n:
                    break
        results.append(current_combo)

        # Update entries to remove used ones if they hit the maximum usage count of 3
        entries = [e for e in entries if usage_count[(e[0], e[1])] < 3]

    return results, usage_count

# Generate round robins
entries = list(df_expanded.itertuples(index=False, name=None))
parlays5, usage5 = create_round_robins(entries, 5, 4)  # 4 groups of 5 teams
parlays4, usage4 = create_round_robins(entries, 4, 4)  # 4 groups of 4 teams
parlays3, usage3 = create_round_robins(entries, 3, 4)  # 4 groups of 3 teams

# Function to calculate parlay odds
def calculate_parlay_odds(combo):
    odds = 1
    for _, _, odds_val, _ in combo:
        odds *= odds_val
    return odds

# Display all parlays
for idx, parlays in enumerate([parlays5, parlays4, parlays3]):
    print(f"Parlays with {5 - idx} teams:")
    for index, combo in enumerate(parlays):
        odds = calculate_parlay_odds(combo)
        print(f"  Round Robin {index + 1} (Odds: {odds}):")
        for game_matchup, edge_type, _, edge_value in combo:
            print(f"    {game_matchup} - {edge_type} Edge: {edge_value}, Odds: {df_expanded.loc[(df_expanded['Game Matchup'] == game_matchup) & (df_expanded['Edge Type'] == edge_type), 'Odds'].iloc[0]}")
        print()


Parlays with 5 teams:
  Round Robin 1 (Odds: 2551.37731875):
    Alaves vs. Girona - Home Edge: 0.44, Odds: 3.7
    Athletic Club vs. Osasuna - Away Edge: 0.31, Odds: 6.25
    Athletic Club vs. Osasuna - Draw Edge: 0.3, Odds: 4.3
    Granada vs. Real Madrid - Home Edge: 0.41, Odds: 5.59
    Granada vs. Real Madrid - Draw Edge: 0.28, Odds: 4.59

  Round Robin 2 (Odds: 2551.37731875):
    Alaves vs. Girona - Home Edge: 0.44, Odds: 3.7
    Athletic Club vs. Osasuna - Away Edge: 0.31, Odds: 6.25
    Athletic Club vs. Osasuna - Draw Edge: 0.3, Odds: 4.3
    Granada vs. Real Madrid - Home Edge: 0.41, Odds: 5.59
    Granada vs. Real Madrid - Draw Edge: 0.28, Odds: 4.59

  Round Robin 3 (Odds: 2551.37731875):
    Alaves vs. Girona - Home Edge: 0.44, Odds: 3.7
    Athletic Club vs. Osasuna - Away Edge: 0.31, Odds: 6.25
    Athletic Club vs. Osasuna - Draw Edge: 0.3, Odds: 4.3
    Granada vs. Real Madrid - Home Edge: 0.41, Odds: 5.59
    Granada vs. Real Madrid - Draw Edge: 0.28, Odds: 4.59

  R

In [4]:
import pandas as pd
import random
from collections import defaultdict, Counter

def distribute_entries(df, parlay_config):
    results = defaultdict(list)
    usage_count = Counter()

    # Create a list of all entries
    entries = list(df.itertuples(index=False, name=None))  # Convert DataFrame to list of tuples

    # Shuffle entries to randomize distribution
    random.shuffle(entries)

    # Prepare a tracker for how many parlays are still needed for each size
    parlay_needs = {size: count for size, count in parlay_config.items()}

    # Loop until all parlay needs are met or entries are exhausted
    while any(parlay_needs.values()) and entries:
        for size in sorted(parlay_needs.keys(), reverse=True):
            if parlay_needs[size] > 0:
                # Attempt to create a parlay of this size
                current_parlay = []
                remaining_entries = []

                for entry in entries:
                    if len(current_parlay) < size and usage_count[entry] < 2:
                        # Check for unique game in the current parlay
                        if all(current[0] != entry[0] for current in current_parlay):
                            current_parlay.append(entry)
                            usage_count[entry] += 1
                        else:
                            remaining_entries.append(entry)
                    else:
                        remaining_entries.append(entry)

                if len(current_parlay) == size:
                    results[size].append(current_parlay)
                    parlay_needs[size] -= 1
                    entries = remaining_entries  # Update entries that are still available

    return results, usage_count



parlay_config = {5: 3, 4: 4, 3: 5}  # Desired number of parlays of each size
parlays, usage_count = distribute_entries(df_expanded, parlay_config)

# Print results
for size, parlay_list in parlays.items():
    print(f"Parlays with {size} teams:")
    for idx, parlay in enumerate(parlay_list):
        print(f"Round Robin {idx + 1}:")
        for entry in parlay:
            print(f"  {entry[0]} - {entry[1]} Edge: {entry[3]}, Odds: {entry[2]}")
        print()

# Print usage counts to verify proper distribution
print("Usage Count:")
for key, count in sorted(usage_count.items()):
    print(f"{key}: included in {count} parlays")


Parlays with 5 teams:
Round Robin 1:
  Athletic Club vs. Osasuna - Away Edge: 0.31, Odds: 6.25
  Frosinone vs. Inter - Away Edge: 0.31, Odds: 1.5917159763313609
  Lazio vs. Empoli - Home Edge: 0.18, Odds: 1.6134969325153374
  Tottenham vs. Burnley - Away Edge: 0.66, Odds: 7.06
  Real Betis vs. Almeria - Away Edge: 0.77, Odds: 7.5

Round Robin 2:
  Freiburg vs. FC Heidenheim - Away Edge: 1.57, Odds: 4.4
  Paris Saint Germain vs. Toulouse - Draw Edge: 0.47, Odds: 5.59
  Atalanta vs. Roma - Home Edge: 0.26, Odds: 2.0
  Tottenham vs. Manchester City - Home Edge: 0.88, Odds: 5.99
  Rennes vs. Lens - Away Edge: 0.16, Odds: 2.75

Round Robin 3:
  Paris Saint Germain vs. Toulouse - Away Edge: 1.15, Odds: 6.0
  Mainz 05 vs. Borussia Dortmund - Home Edge: 0.2, Odds: 2.25
  Bochum vs. Bayer Leverkusen - Away Edge: 0.28, Odds: 1.6944444444444444
  Borussia M.Gladbach vs. Eintracht Frankfurt - Home Edge: 0.5, Odds: 2.5
  Newcastle United vs. Brighton - Home Edge: 0.39, Odds: 1.6134969325153374

Par

In [5]:
import pandas as pd
import random
from collections import defaultdict, Counter

def distribute_entries(df, parlay_config):
    results = defaultdict(list)
    usage_count = Counter()

    # Create a list of all entries
    entries = list(df.itertuples(index=False, name=None))  # Convert DataFrame to list of tuples

    # Shuffle entries to randomize distribution
    random.shuffle(entries)

    # Prepare a tracker for how many parlays are still needed for each size
    parlay_needs = {size: count for size, count in parlay_config.items()}

    # Loop until all parlay needs are met or entries are exhausted
    while any(parlay_needs.values()) and entries:
        for size in sorted(parlay_needs.keys(), reverse=True):
            if parlay_needs[size] > 0:
                # Attempt to create a parlay of this size
                current_parlay = []
                remaining_entries = []

                for entry in entries:
                    if len(current_parlay) < size and usage_count[entry] < 2:
                        # Check for unique game in the current parlay
                        if all(current[0] != entry[0] for current in current_parlay):
                            current_parlay.append(entry)
                            usage_count[entry] += 1
                        else:
                            remaining_entries.append(entry)
                    else:
                        remaining_entries.append(entry)

                if len(current_parlay) == size:
                    results[size].append(current_parlay)
                    parlay_needs[size] -= 1
                    entries = remaining_entries  # Update entries that are still available

    return results, usage_count

parlay_config = {5: 3, 4: 4, 3: 5}  # Desired number of parlays of each size
parlays, usage_count = distribute_entries(df_expanded, parlay_config)

# Print results
for size, parlay_list in parlays.items():
    print(f"Parlays with {size} teams:")
    for idx, parlay in enumerate(parlay_list):
        print(f"Round Robin {idx + 1}:")
        for entry in parlay:
            print(f"  {entry[0]} - {entry[1]} Edge: {entry[3]}, Odds: {entry[2]}")
        print()

# Print usage counts to verify proper distribution
print("Usage Count:")
for key, count in sorted(usage_count.items()):
    print(f"{key}: included in {count} parlays")


Parlays with 5 teams:
Round Robin 1:
  Genoa vs. Sassuolo - Home Edge: 0.61, Odds: 2.2
  Nottingham Forest vs. Chelsea - Home Edge: 0.65, Odds: 3.49
  Mainz 05 vs. Borussia Dortmund - Home Edge: 0.2, Odds: 2.25
  Atalanta vs. Roma - Home Edge: 0.26, Odds: 2.0
  Fiorentina vs. Monza - Away Edge: 0.19, Odds: 5.4

Round Robin 2:
  AC Milan vs. Cagliari - Draw Edge: 0.18, Odds: 5.2
  Mallorca vs. Las Palmas - Draw Edge: 0.23, Odds: 3.6
  FC Cologne vs. Union Berlin - Away Edge: 0.36, Odds: 2.5
  Real Betis vs. Almeria - Draw Edge: 0.62, Odds: 5.4
  Bournemouth vs. Brentford - Away Edge: 0.24, Odds: 3.49

Round Robin 3:
  Rennes vs. Lens - Away Edge: 0.16, Odds: 2.75
  Fulham vs. Manchester City - Home Edge: 1.74, Odds: 12.84
  Mallorca vs. Las Palmas - Away Edge: 0.38, Odds: 6.0
  Athletic Club vs. Osasuna - Away Edge: 0.31, Odds: 6.25
  Alaves vs. Girona - Home Edge: 0.44, Odds: 3.7

Parlays with 4 teams:
Round Robin 1:
  Tottenham vs. Manchester City - Home Edge: 0.88, Odds: 5.99
  Paris

In [ ]:
from itertools import combinations

def calculate_round_robin_payouts(parlays, stake):
    payouts = {}

    for size, parlay_list in parlays.items():
        payouts[size] = []
        total_potential_winnings = 0
        total_risk = 0

        for parlay in parlay_list:
            parlay_odds = 1
            for entry in parlay:
                parlay_odds *= entry[2]  # Multiply the odds for each team in the parlay

            potential_winnings = stake * parlay_odds
            risk = stake
            payouts[size].append((potential_winnings, risk))
            total_potential_winnings += potential_winnings
            total_risk += risk

            # Calculate payouts for individual parlays within the current parlay
            for combo_size in range(2, size + 1):
                for combo in combinations(parlay, combo_size):
                    combo_odds = 1
                    for entry in combo:
                        combo_odds *= entry[2]
                    combo_potential_winnings = stake * combo_odds
                    print(f"Parlay: {' - '.join(entry[0] for entry in combo)}, Potential Winnings: ${combo_potential_winnings:.2f}, Risk: ${stake}")
                    total_potential_winnings += combo_potential_winnings
                    total_risk += stake

        payouts[size].append(("Total Potential Winnings", total_potential_winnings))
        payouts[size].append(("Total Risk", total_risk))

    return payouts

stake = 10
payouts = calculate_round_robin_payouts(parlays, stake)

# Print payouts
for size, parlay_payouts in payouts.items():
    print(f"\nParlays with {size} teams:")
    for payout in parlay_payouts:
        if isinstance(payout[0], str):
            print(f"{payout[0]}: ${payout[1]:.2f}")
        else:
            print(f"Potential Winnings: ${payout[0]:.2f}, Risk: ${payout[1]}")

In [ ]:
from itertools import combinations

def calculate_round_robin_payouts(parlay, stake):
    total_potential_winnings = 0
    total_risk = 0

    # Calculate the total odds for the 5-team parlay
    total_odds = 1
    for entry in parlay:
        total_odds *= entry[2]

    potential_winnings = stake * total_odds
    total_potential_winnings += potential_winnings
    total_risk += stake

    # Calculate payouts for individual parlays
    # Pairs (2-team parlays)
    for pair in combinations(parlay, 2):
        parlay_odds = pair[0][2] * pair[1][2]
        potential_winnings = stake * parlay_odds
        print(f"Parlay: {pair[0][0]} - {pair[1][0]}, Potential Winnings: ${potential_winnings:.2f}, Risk: ${stake}")
        total_potential_winnings += potential_winnings
        total_risk += stake

    # Trios (3-team parlays)
    for trio in combinations(parlay, 3):
        parlay_odds = trio[0][2] * trio[1][2] * trio[2][2]
        potential_winnings = stake * parlay_odds
        print(f"Parlay: {trio[0][0]} - {trio[1][0]} - {trio[2][0]}, Potential Winnings: ${potential_winnings:.2f}, Risk: ${stake}")
        total_potential_winnings += potential_winnings
        total_risk += stake

    # Quartets (4-team parlays)
    for quartet in combinations(parlay, 4):
        parlay_odds = quartet[0][2] * quartet[1][2] * quartet[2][2] * quartet[3][2]
        potential_winnings = stake * parlay_odds
        print(f"Parlay: {quartet[0][0]} - {quartet[1][0]} - {quartet[2][0]} - {quartet[3][0]}, Potential Winnings: ${potential_winnings:.2f}, Risk: ${stake}")
        total_potential_winnings += potential_winnings
        total_risk += stake

    return total_potential_winnings, total_risk

# Assuming you have the first 5-team parlay from your 'parlays' dictionary
first_5_team_parlay = parlays[5][0]
stake = 10

total_potential_winnings, total_risk = calculate_round_robin_payouts(first_5_team_parlay, stake)

print(f"\nTotal Potential Winnings: ${total_potential_winnings:.2f}, Total Risk: ${total_risk}")